In [8]:
from sklearn.decomposition import PCA
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import numpy as np
import onnxruntime as rt

# Train a PCA model.
X = np.array([[0., 0., 3.], [1., 1., 0.], [0., 2., 1.], [1., 0., 2.]])
pca = PCA(n_components=2)
pca.fit(X)
x_transform = pca.transform(X)

In [1]:

# Convert to ONNX format.
initial_type = [('float_input', FloatTensorType([None, X.shape[1]]))]
onnx_model = convert_sklearn(pca, initial_types=initial_type)

# Save to a file.
with open("pca.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [5]:

# Load and run the model using ONNX Runtime.
sess = rt.InferenceSession("pca.onnx")
input_name = sess.get_inputs()[0].name
test = X.astype(np.float32)
test

array([[0., 0., 3.],
       [1., 1., 0.],
       [0., 2., 1.],
       [1., 0., 2.]], dtype=float32)

In [14]:
test_2 = np.array([[0., 2., 1.], [1., 1., 0.], [0., 2., 1.], [1., 0., 2.]])
test_v2 = test_2.astype(np.float32)
test_v2

array([[0., 2., 1.],
       [1., 1., 0.],
       [0., 2., 1.],
       [1., 0., 2.]], dtype=float32)

In [15]:
pred_onx = sess.run(None, {input_name: test_v2})[0]
print(pred_onx)


[[-1.0274334   1.0000001 ]
 [-1.4498659  -0.6666667 ]
 [-1.0274334   1.0000001 ]
 [ 0.76491034 -0.6666667 ]]


In [10]:
from sklearn.metrics import accuracy_score
# print(accuracy_score(x_transform,pred_onx)) 
x_transform

array([[ 1.71238903,  0.33333333],
       [-1.44986595, -0.66666667],
       [-1.02743342,  1.        ],
       [ 0.76491034, -0.66666667]])

In [11]:
# identical as x_transform
pred_onx 

array([[ 1.7123891 ,  0.33333334],
       [-1.4498659 , -0.6666667 ],
       [-1.0274334 ,  1.0000001 ],
       [ 0.76491034, -0.6666667 ]], dtype=float32)

In [17]:
pca2 = PCA(n_components=2)
pca2.fit(test_2)
x_transform2 = pca2.transform(X)
x_transform2

array([[ 1.63299316e+00, -1.69118865e+00],
       [ 3.88578059e-16,  1.14564392e+00],
       [-8.16496581e-01, -3.81881308e-01],
       [ 1.63299316e+00, -3.81881308e-01]])